In [27]:
!pip install colabcode
!pip install fastapi
!pip install pydantic
!pip install pyngrok
!pip install firebase_admin
!pip install numpy
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nbclassic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

# Firebase

In [13]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Khởi tạo kết nối với Firebase
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://coffee-4053c-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [12]:
def GetProducts():
    # Lấy dữ liệu từ nút "SanPham" trong Firebase
    product_data = db.reference('SanPham').get()

    # Lấy dữ liệu từ nút "LoaiSanPham" trong Firebase
    product_type_data = db.reference('LoaiSanPham').get()
    list_product_type = list(product_type_data.values())

    # Xử lý dữ liệu
    products = [
        {
            'MaSanPham': product['MaSanPham'],
            'TenSanPham': product['TenSanPham'],
            'SoLuong': product['SoLuong'],
            'MoTa': product['Mota'],
            'HinhAnh': product['HinhAnh'],
            'LoaiSanPham': next((x['LoaiSanPham'] for x in list_product_type if x['MaLoaiSanPham'] == product['MaLoaiSanPham']), None),
            'CongThuc': [congthuc['TenNguyenLieu'] for congthuc in product['CongThuc'].values()]
        }
        for product in product_data.values()
    ]

    return products

In [14]:
from collections import Counter

def GetTopProductID(number = 5):
  bill_data = db.reference('HoaDon').get()

  product_quantities = Counter()

  for bill_details in bill_data.values():
      for product_details in bill_details['ChiTietHoaDon'].values():
          product_quantities[product_details['MaSanPham']] += product_details['SoLuong']

  top_products = product_quantities.most_common(number)

  return [productID for productID, quantity in top_products]

In [15]:
products = GetProducts()
product_size = len(products)

# Chatbot

In [2]:
!pip install tensorflow
!pip install transformers

In [1]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout, Input, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
bert_name = "vinai/phobert-base"

# Load the pre-trained sentence embedding model
bert = TFAutoModel.from_pretrained(bert_name)
tokenizer = AutoTokenizer.from_pretrained(bert_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [3]:
model = load_model('model_coffee_question_classification.h5')

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/output/dense/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'tf_roberta_model/roberta/encoder/layer_.

In [4]:
MAX_LEN_CONTEXT = 23

def getNumberPredict(question):
  test_contexts = [question]

  test_context_encodings = tokenizer(test_contexts, truncation=True, padding='max_length', max_length=MAX_LEN_CONTEXT)

  test_context_features = {key: tf.convert_to_tensor(test_context_encodings[key], dtype=tf.float32) for key in tokenizer.model_input_names}

  test_context_input_ids = test_context_features['input_ids']
  test_context_attention_mask = test_context_features['attention_mask']

  Test_array = model.predict([test_context_input_ids, test_context_attention_mask])

  y_predict = np.argmax(Test_array, axis=-1)

  return y_predict[0]

In [5]:
def extract_number_from_id(product_id):
    # Lấy phần số từ mã sản phẩm dạng "SP0001"
    return int(product_id[2:])

In [6]:
def binary_search(product_id):
    left, right = 0, product_size - 1
    while left <= right:
        mid = (left + right) // 2
        mid_id = extract_number_from_id(products[mid]['MaSanPham'])
        target_id = extract_number_from_id(product_id)
        if mid_id == target_id:
            return products[mid]
        elif mid_id < target_id:
            left = mid + 1
        else:
            right = mid - 1
    return None  # Không tìm thấy sản phẩm

In [7]:
def computeLPSArray(pattern, n):
    length = 0  # Độ dài của tiền tố dài nhất mà cũng là hậu tố
    lps = [0] * n  # Khởi tạo bảng LPS
    i = 1

    while i < n:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def KMPSearch(text, n, pattern, lps):
    m = len(pattern)

    if (m > n):
      return False

    i = 0  # Chỉ số cho text
    j = 0  # Chỉ số cho pattern

    while i < n:
        if pattern[j] == text[i]:
            i += 1
            j += 1

        if j == m:
            return True  # Tìm thấy pattern trong text
        elif i < n and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return False  # Không tìm thấy pattern trong text

In [8]:
def infomationProduct(product):
  text = f"{product['TenSanPham']} thuộc loại sản phảm {product['LoaiSanPham']} được mô tả mô tả {product['MoTa']} được chế biến bởi những nguyên liệu "

  for congthuc in product['CongThuc']:
    text += f"{congthuc}, "

  text = text + f"và hiện tại có số lượng là {product['SoLuong']}.\n"

  return text

In [9]:
import requests

url = "http://huynhnhan2003.pythonanywhere.com/recommend"

In [10]:
beginMessage = "Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo. "
endMessage = "Mong bạn hài lòng với câu trả lời của chúng tôi.\nHiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, sản phẩm bán chạy và gợi ý sản phẩm.\nVí dụ:\n- Cà phê sữa còn hàng không?\n- Cho tôi danh sách sản phẩm bán chạy của quán.\n- Gợi ý cho tôi sản phẩm với.\nChân thành cảm ơn bạn đã sử dụng dịch vụ của quán."

In [36]:
def getMessage(number, question, customer_id):
  message = beginMessage

  question = question.lower()
  n = len(question)

  lps = computeLPSArray(question, n)

  productInQuestion = []

  haveNgon = False

  if KMPSearch(question, n, "ngon", lps):
    haveNgon = True
    message = message + "Sản phẩm ở quán đều ngon để phục vụ quý khách.\n"

  if number == 0 or number == 2:
    for product in products:
      if KMPSearch(question, n, product['TenSanPham'].lower(), lps):
        productInQuestion.append(product)

  sizeProductInQuestion = len(productInQuestion)

  if number == 0 and sizeProductInQuestion == 0:
    number = 2

  if number == 0:
    # information
    if not haveNgon:
      message = message + "\n"

    for product in productInQuestion:
      message = message + infomationProduct(product)

  elif number == 1:
    # top sell
    productIDs = GetTopProductID()

    productTopSell = [binary_search(productID) for productID in productIDs]

    message = message + 'Sản phẩm được bán chạy nhiều nhất của cửa hàng:\n'

    for product in productTopSell:
      message += f'- {product["TenSanPham"]}\n'

  elif number == 2:
    # recommend
    payload = {
        "MaSanPham": "SP0001",
        "MaKhachHang": customer_id
    }

    if sizeProductInQuestion != 0:
      payload['MaSanPham'] = productInQuestion[0]['MaSanPham']
      payload['MaKhachHang'] = ""

    response = requests.post(url, json=payload)

    productsRecommend = response.json()

    message = message + 'Sản phẩm gợi ý cho bạn:\n'

    for productRecommend in productsRecommend:
      message += f'- {productRecommend["TenSanPham"]}\n'

  return message + endMessage

# API

In [16]:
def chatBotGetMessage(question, customer_id):
  number = getNumberPredict(question)

  return getMessage(number, question, customer_id)

In [24]:
print(chatBotGetMessage("Cà phê nào ngon", "KH0001"))

1/1 [==============================] - 0s 61ms/step
Chào bạn, ESPRO xin trả lời câu hỏi của bạn lưu ý đây là tin nhắn tự động của trí tuệ nhân tạo. Sản phẩm được phù hợp với bạn:
- Latte kẹo đường
- Latte đường nâu
- Cà phê Espresso
- Cà phê Cappuccino
- Cà phê Mocha
Mong bạn hài lòng với câu trả lời của chúng tôi.
Hiện tại, chúng tôi chỉ trả lời các câu hỏi liên quan đến thông tin sản phẩm, sản phẩm bán chạy và gợi ý sản phẩm.
Ví dụ:
- Cà phê sữa còn hàng không?
- Cho tôi danh sách sản phẩm bán chạy của quán.
- Gợi ý cho tôi sản phẩm với.
Chân thành cảm ơn bạn đã sử dụng dịch vụ của quán.


In [28]:
from colabcode import ColabCode
from fastapi import FastAPI
from pydantic import BaseModel
import json
from pyngrok import ngrok

In [34]:
!ngrok config add-authtoken 2RTXfbCm3Gm5DCFEFfHAprwXgdI_3HPFH8ExdVKq4631Gixxp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [29]:
cc = ColabCode(port=12000, code=False)

In [30]:
app = FastAPI()

@app.get("/")
async def home():
  return {"Hello API Coffee"}

In [31]:
from pydantic import BaseModel

class requestChatbot(BaseModel):
    MaKhachHang : str
    CauHoi : str

In [32]:
@app.post("/chatbot")
async def reply(request: requestChatbot):
    try:
      request_data = request.json()
      request_dictionary = json.loads(request_data)

      user_id = request_dictionary['MaKhachHang']
      question = request_dictionary['CauHoi']

      return {chatBotGetMessage(question, user_id)}
    except:
        return {"Message": "error"}

In [ ]:
cc.run_app(app=app)